In [1]:
data_dir = 'C:/Users/maruv/Desktop/DSB/LDA/'

In [2]:
import json
import os
import glob

In [3]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
df_electronics = getDF(data_dir+'reviews_Electronics_5.json.gz')

In [5]:
df_electronics.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [11]:
from scipy.stats import itemfreq

In [12]:
item_id = df_electronics.as_matrix(columns=df_electronics.columns[1:2])
freq_items =itemfreq(item_id)

In [14]:
freq_df = pd.DataFrame(data=freq_items,columns=['item','freq'])

In [15]:
freq_df.head()

,item,freq
0,0528881469,5
1,0594451647,5
2,0594481813,8
3,0972683275,219
4,1400501466,43


In [16]:
freq_df.loc[freq_df['freq'].idxmax()]

item    B007WTAJTO
freq          4915
Name: 49156, dtype: object

In [18]:
max_reviewed_item = df_electronics.loc[df_electronics.asin.isin(['B007WTAJTO']),['reviewText']]

In [19]:
max_reviewed_item.head()

,reviewText
1336614,No issues.
1336615,"Purchased this for my device, it worked as adv..."
1336616,it works as expected. I should have sprung for...
1336617,This think has worked out great.Had a diff. br...
1336618,"Bought it with Retail Packaging, arrived legit..."


In [20]:
final_max_corp = max_reviewed_item['reviewText']

In [21]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_max_clean = [clean(doc).split() for doc in final_max_corp]

In [22]:
import gensim
from gensim import corpora

# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_max_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_max_term_matrix = [dictionary.doc2bow(doc) for doc in doc_max_clean]

C:\Users\maruv\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [23]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_max_term_matrix, num_topics=5, id2word = dictionary, passes=50)

In [24]:
print(ldamodel.print_topics(num_topics=5, num_words=3))

[(0, '0.063*"galaxy" + 0.043*"samsung" + 0.034*"note"'), (1, '0.030*"card" + 0.022*"work" + 0.021*"phone"'), (2, '0.038*"card" + 0.022*"camera" + 0.020*"work"'), (3, '0.053*"card" + 0.015*"sandisk" + 0.014*"phone"'), (4, '0.048*"card" + 0.018*"sd" + 0.016*"phone"')]
